In [1]:
import os
import pickle
import glob
import gzip
from tqdm.notebook import tqdm #from tqdm import tqdm_notebook as tqdm
import pandas as pd
import datetime

In [2]:
path_0 = "./folder_01"
path_1 = '../folder_01' #input
path_2 = '../folder_03' #output

In [3]:
#2021_mesh
os.chdir('./')
filename_mesh = 'mesh_list_2021.pickle'
with open(filename_mesh, 'rb') as f:
    mesh_list = pickle.load(f)
    
all_disease = [descriptor for descriptor in mesh_list for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
all_symptom = [descriptor for descriptor in mesh_list  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]

#重複削除
disease_mesh = []
[dm for dm in all_disease if dm not in disease_mesh and not disease_mesh.append(dm)]
symptom_mesh = []
[ms for ms in all_symptom if ms not in symptom_mesh and not symptom_mesh.append(ms)]

print("all_mesh:", len(mesh_list), "disease_mesh:", len(disease_mesh), "symptom_mesh:", len(symptom_mesh))

#ターム名だけ抽出
disease_mesh_name_list = [i[1] for i in disease_mesh]
symptom_mesh_name_list = [i[1] for i in symptom_mesh]

all_mesh: 29917 disease_mesh: 4887 symptom_mesh: 394


In [5]:
#PubMed文献データcsvから症状を取り出す関数
def select_decriptor_symptom(df):

    if len(df)>0:#csvにデータがある場合
        
        #症状にあたるMeSHを抽出
        s_index_num = [enu for enu, i in enumerate(df["Descriptor"]) if i in symptom_mesh_name_list] 
        tmp_df = df.iloc[s_index_num]#.reset_index(drop=True)
        
        #症状メッシュはMajorTopicで絞り込む必要はないのでそのまま用いる
        tmp_df = tmp_df.reset_index(drop=True)

        #make csv
        os.chdir(path_2)
        csvname = "s_"+file[18:-4]+".csv"
        tmp_df.to_csv(csvname, index=False)

    else:#csvにデータがない場合
        tmp_df = pd.DataFrame(columns=["PMID","PubDate",
                                       "Descriptor","Descriptor_UI","Descriptor_MajorTopic",
                                       "Qualifier","Qualifier_UI","Qualifier_MajorTopic",
                                      ])
        
        #make csv
        os.chdir(path_2)
        csvname = "s_"+file[18:-4]+".csv"
        tmp_df.to_csv(csvname, index=False)  

    os.chdir(path_1) 
    return

In [7]:
# PubMed文献データcsvをリスト化
os.chdir(path_0)
csv_list = sorted(glob.glob('one_desc_one_qual_*.csv'))
print(len(csv_list), csv_list[:1])

1 ['one_desc_one_qual_0001.csv']


In [8]:
#folder_03 に症状CSV作成
for file in tqdm(csv_list[:]):
    df = pd.read_csv(file, engine='python') 
    select_decriptor_symptom(df)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#症状データの抽出完了